# COSMOS master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table, Column

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180516"
FIELD = "COSMOS"
catname = "../../dmu1/dmu1_ml_COSMOS/data/master_catalogue_cosmos_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
u_bands = ["Megacam u",                      "Omegacam u"]
g_bands = ["Megacam g", "DECam g", "GPC1 g", "Omegacam g", "SUPRIME g"]
r_bands = ["Megacam r", "DECam r", "GPC1 r", "Omegacam r", "SUPRIME r"]
i_bands = ["Megacam i",            "GPC1 i", "Omegacam i", "SUPRIME i"]
z_bands = ["Megacam z", "DECam z", "GPC1 z",           "SUPRIME z"]
y_bands = [                        "GPC1 y",           "SUPRIME y", "UKIDSS Y"]
J_bands = [                                                         "UKIDSS J", "WIRCAM J"]
H_bands = [                                                         "UKIDSS H", "WIRCAM H"]
K_bands = [                                                         "UKIDSS K", "WIRCAM Ks"]

all_bands = [u_bands, g_bands, r_bands, i_bands, z_bands, y_bands, J_bands, H_bands, K_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue.add_column(Column(data=np.zeros(len(master_catalogue), dtype=bool), name=flagcol))
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry cat 
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_ap_{}".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

GPC1 g
  Aperture magnitude
    Number of flagged objects: 60
  Total magnitude
    Number of flagged objects: 60
GPC1 r
  Aperture magnitude
    Number of flagged objects: 100
  Total magnitude
    Number of flagged objects: 99
GPC1 i
  Aperture magnitude
    Number of flagged objects: 139
  Total magnitude
    Number of flagged objects: 159
GPC1 z
  Aperture magnitude
    Number of flagged objects: 69
  Total magnitude
    Number of flagged objects: 67
GPC1 y
  Aperture magnitude
    Number of flagged objects: 36
  Total magnitude
    Number of flagged objects: 39


### 2.a DECaLS Total magnitudes

In [6]:
decam_mag = 14.999935
bands = ["DECam g", "DECam r", "DECam z"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    col = "m_{}".format(basecol)
    flagcol = "flag_{}".format(basecol)
    
    mask  = np.where((master_catalogue[col]  == decam_mag) | (master_catalogue[col] < 7))[0]
    
    print('  Total magnitude')
    flag_mag(flagcol, mask)

DECam g
  Total magnitude
    Number of flagged objects: 0
DECam r


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


  Total magnitude
    Number of flagged objects: 1
DECam z
  Total magnitude
    Number of flagged objects: 0


## 2. Outliers

In [8]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):
        #print(band1, band2)

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))

Omegacam u (aperture) - Megacam u (aperture):
  Number of outliers: 0
Omegacam u (total) - Megacam u (total):
  Number of outliers: 24
DECam g (aperture) - Megacam g (aperture):
  Number of outliers: 4
DECam g (total) - Megacam g (total):
  Number of outliers: 100


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 80697 but corresponding boolean dimension is 80696


GPC1 g (aperture) - Megacam g (aperture):
  Number of outliers: 0


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 24320 but corresponding boolean dimension is 24319


GPC1 g (total) - Megacam g (total):
  Number of outliers: 0


...

SUPRIME z (aperture) - GPC1 z (aperture):
  Number of outliers: 0
SUPRIME z (total) - GPC1 z (total):
  Number of outliers: 0
SUPRIME y (aperture) - GPC1 y (aperture):
  Number of outliers: 0
SUPRIME y (total) - GPC1 y (total):
  Number of outliers: 0


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 108947 but corresponding boolean dimension is 108946


UKIDSS Y (aperture) - GPC1 y (aperture):
  Number of outliers: 13
UKIDSS Y (total) - GPC1 y (total):
  Number of outliers: 16
UKIDSS Y (aperture) - SUPRIME y (aperture):
  Number of outliers: 26
UKIDSS Y (total) - SUPRIME y (total):
  Number of outliers: 8
WIRCAM J (aperture) - UKIDSS J (aperture):
  Number of outliers: 0
WIRCAM J (total) - UKIDSS J (total):
  Number of outliers: 1
WIRCAM H (aperture) - UKIDSS H (aperture):
  Number of outliers: 1
WIRCAM H (total) - UKIDSS H (total):
  Number of outliers: 3
WIRCAM Ks (aperture) - UKIDSS K (aperture):
  Number of outliers: 0


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel/__main__.py:54: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 17173 but corresponding boolean dimension is 17172


WIRCAM Ks (total) - UKIDSS K (total):
  Number of outliers: 0


## 3. Save table

In [ ]:

#Merge any aperture flags#Merge  
for col in master_catalogue.colnames:
    if col.startswith("flag_ap_"):
        try:
            master_catalogue[col.replace("_ap_", "_")] = (master_catalogue[col.replace("_ap_", "_")] |
                                                          master_catalogue[col])
            master_catalogue.remove_column(col)
        except KeyError:
            print("{} only has aperture flags.".format(col))
            master_catalogue.rename_column(col, col.replace("_ap_", "_"))

In [9]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)